# anakin87/electra-italian-xxl-cased-squad-it
# Usage in Haystack 🚀🚀🚀

With the [Haystack NLP framework](https://github.com/deepset-ai/haystack), you can use this model and create a scalable Question Answering system that works across millions of documents.

This notebook is inspired by [the first Haystack tutorial](https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline/).


## Installing Haystack



In [ ]:
! pip install --upgrade pip && pip install farm-haystack[colab]

## Loading and preparing some documents from the fanpage dataset

In [ ]:
! pip install datasets

In [5]:
from datasets import load_dataset
from haystack import Document
from haystack.nodes import PreProcessor


dataset = load_dataset("ARTeLab/fanpage", split='train[:1000]')

In [6]:
docs = [Document(content=el) for el in dataset['source']]

In [7]:
len(docs)

1000

In [8]:
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
    language="it"
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
preprocessed_docs = preprocessor.process(docs)

Preprocessing:   0%|          | 0/1000 [00:00<?, ?docs/s]

In [10]:
len(preprocessed_docs)

2307

## Initializing the DocumentStore


In [11]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

In [12]:
# write the documents
document_store.write_documents(preprocessed_docs)

Updating BM25 representation...:   0%|          | 0/2307 [00:00<?, ? docs/s]

## Initializing the Retriever

In [13]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

## Initializing the Reader

It is based on `anakin87/electra-italian-xxl-cased-squad-it`

In [14]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="anakin87/electra-italian-xxl-cased-squad-it", use_gpu=True)

## Creating the Retriever-Reader Pipeline

In [15]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

## Asking a Question

In [21]:
prediction = pipe.run(
    query="Con chi ha parlato di vaccini il premier Mario Draghi?",
    params={
        "Retriever": {"top_k": 5},
        "Reader": {"top_k": 3}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [23]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="medium" ## Choose from `minimum`, `medium`, and `all`
)


Query: Con chi ha parlato di vaccini il premier Mario Draghi?
Answers:
[   {   'answer': 'Von der Leyen',
        'context': " vaccino dell'azienda britannica. Durante la telefonata "
                   'tra Draghi e Von der Leyen, la presidente della '
                   'Commissione Ue ha annunciato al presidente del',
        'score': 0.9663902521133423},
    {   'answer': 'Ursula Von der Leyen',
        'context': 'colloquio telefonico con la presidente della Commissione '
                   'europea Ursula Von der Leyen. Secondo fonti di Palazzo '
                   'Chigi, dalla conversazione è emerso ch',
        'score': 0.9063920974731445},
    {   'answer': 'Mario Draghi, ha tenuto un lungo discorso alla 76esima '
                  'Assemblea Generale delle Nazioni Unite',
        'context': 'Il presidente del Consiglio, Mario Draghi, ha tenuto un '
                   'lungo discorso alla 76esima Assemblea Generale delle '
                   'Nazioni Unite, nella notte itali

In [18]:
prediction = pipe.run(
    query="Che lavoro fa Maurizio Sarri?",
    params={
        "Retriever": {"top_k": 5},
        "Reader": {"top_k": 3}
    }
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [19]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="medium" ## Choose from `minimum`, `medium`, and `all`
)


Query: Che lavoro fa Maurizio Sarri?
Answers:
[   {   'answer': 'allenatore',
        'context': 'Da qualsiasi parte la si guardi, la prima stagione da '
                   'allenatore di Andrea Pirlo non è stata esattamente un '
                   'successo. E poiché la panchina su cui si è',
        'score': 0.9699883460998535},
    {   'answer': 'docente di scrittura',
        'context': 'onne", un\'autrice molto prolifica e dalla penna raffinata '
                   'nonché docente di scrittura da moltissimi anni. A '
                   "quest'ultima abbiamo chiesto di rispondere",
        'score': 0.7437450885772705},
    {   'answer': 'allentatore',
        'context': ' è nata questa parola abbia strettamente a che fare col '
                   "mestiere dell'allentatore, essendo un significato tecnico, "
                   'di metodo: proprio come chiosa la T',
        'score': 0.7240688800811768}]
